<a href="https://colab.research.google.com/github/tagtog12000/GrandPotentialPython/blob/main/eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import ast
import subprocess
import os

REPO_NAME = "GrandPotentialPython"
REPO_URL = f"https://github.com/tagtog12000/GrandPotentialPython.git"

# --- 1. Clone the GitHub repo ---
if not os.path.exists(REPO_NAME):
    print(f"🔽 Cloning repository: {REPO_URL}")
    subprocess.run(["git", "clone", REPO_URL], check=True)
else:
    print("✅ Repository already cloned.")

# --- 2. Change directory into the repo ---
os.chdir(REPO_NAME)

# --- 3. Install dependencies if available ---
if os.path.exists("requirements.txt"):
    print("📦 Installing Python dependencies ...")
    subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)
else:
    print("⚠️ No requirements.txt found — skipping dependency install.")

# --- 4. Compile the C++ file if not already compiled ---
if not os.path.exists("code"):
    print("⚙️ Compiling GP.cpp ...")
    subprocess.run(["g++", "GP.cpp", "-o", "code"], check=True)
else:
    print("✅ 'code' executable already exists.")

# --- 5. Run Section ---
nMax = int(input("Enter the maximum order nMax: "))
eps = int(input("Enter the particle statistics, eps (-1 for fermions and 1 for bosons): "))

try:
    # Run the compiled executable with both arguments
    result = subprocess.run(
        ["./code", str(nMax), str(eps)],
        capture_output=True, text=True, check=True
    )
    print("✅ Program output:")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("❌ Error while running executable:")
    print(e.stderr)

import loadCode

allData = []  # list to hold data for all n
for n in range(2, nMax+1):
    data_n = {}  # dictionary to hold data for this specific n

    with open(f"matCoefF-{n}.dat") as f:
        data_n["matCoefF"] = ast.literal_eval(f.read())

    with open(f"matDenNigF-{n}.dat") as f:
        data_n["matDenNigF"] = ast.literal_eval(f.read())

    with open(f"matNumSignF-{n}.dat") as f:
        data_n["matNumSignF"] = ast.literal_eval(f.read())

    with open(f"graphs-{n}.dat") as f:
        data_n["graphs"] = ast.literal_eval(f.read())

    with open(f"spanningTrees-{n}.dat") as f:
        data_n["spanningTrees"] = ast.literal_eval(f.read())

    with open(f"symmetries-{n}.dat") as f:
        data_n["symmetries"] = ast.literal_eval(f.read())

    with open(f"refDenominators-{n}.dat") as f:
        data_n["refDenominators"] = ast.literal_eval(f.read())

    with open("DivDifMB-"+str(n)+".dat") as f:
         data_n["DivDifMB"] = ast.literal_eval(f.read())

    dn = 2*n
    ones = (1 << (dn))-1

    loadCode.build_latex(data_n["matCoefF"], data_n["matNumSignF"], data_n["matDenNigF"], data_n["graphs"], data_n["symmetries"], data_n["spanningTrees"], data_n["refDenominators"], ones, n)

🔽 Cloning repository: https://github.com/tagtog12000/GrandPotentialPython.git
📦 Installing Python dependencies ...
⚙️ Compiling GP.cpp ...
Enter the maximum order nMax: 4
Enter the particle statistics, eps (-1 for fermions and 1 for bosons): -1
✅ Program output:



NameError: name 'HTML' is not defined